In [ ]:
#Author: Anh Phan (GitHub: @anhphan1, Email: anhphan@udel.edu, Phone: (302) 345-5885)

%pip install yfinance
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
%pip install keras
%pip install tensorflow
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
yf.pdr_override()

In [ ]:
stock = 'AAPL'
data = yf.download(stock, start='2020-01-01', end='2022-01-01')
data = data[['Close']]

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Historical Stock Price of ' + stock)
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.plot(data['Close'])
plt.show()

In [5]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data)

In [6]:
train_data = scaled_data[0:int(len(scaled_data)*0.95), :]
x_train, y_train = [], []
for i in range(240, len(train_data)):
    x_train.append(train_data[i-240:i, 0])
    y_train.append(train_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:
from keras.callbacks import EarlyStopping

class StockLSTMTrainer:

    # ... [rest of the class remains unchanged]

    def preprocess_data(self, window_size=240, validation_split=0.1):
        self.scaled_data = self.scaler.fit_transform(self.data)
        x, y = [], []
        for i in range(window_size, len(self.scaled_data)):
            x.append(self.scaled_data[i-window_size:i, 0])
            y.append(self.scaled_data[i, 0])
        x, y = np.array(x), np.array(y)
        x = np.reshape(x, (x.shape[0], x.shape[1], 1))
        
        # Splitting into training and validation sets
        val_size = int(validation_split * len(x))
        x_train, x_val = x[:-val_size], x[-val_size:]
        y_train, y_val = y[:-val_size], y[-val_size:]
        
        return (x_train, y_train), (x_val, y_val)

    def train(self, epochs=10, batch_size=64):
        (x_train, y_train), (x_val, y_val) = self.preprocess_data()
        if not self.model:
            self.build_model(input_shape=(x_train.shape[1], 1))

    
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        
        self.model.fit(x_train, y_train, 
                       epochs=epochs, 
                       batch_size=batch_size,
                       validation_data=(x_val, y_val),
                       callbacks=[early_stop])

In [ ]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=12, batch_size=70)

In [8]:
test_data = scaled_data[int(len(scaled_data)*0.95)-60:, :]
x_test, y_test = [], data[int(len(scaled_data)*0.95):].values
for i in range(240, len(test_data)):
    x_test.append(test_data[i-240:i, 0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(np.reshape(predictions, (predictions.shape[0], 1)))

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Prediction vs Real Stock Price of ' + stock)
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.plot(y_test, label='True Price')
plt.plot(predictions, label='Predicted Price')
plt.legend()
plt.show()

In [11]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("The Root Mean Squared Error (RMSE) is:", rmse)

The Root Mean Squared Error (RMSE) is: 13.171473003618004


In [12]:
forecast_days = 30
input_seq = scaled_data[-60:].tolist()

In [ ]:
for _ in range(forecast_days):

    next_input = np.array(input_seq[-60:])
    next_input = np.reshape(next_input, (1, 60, 1))
    
    predicted_price = model.predict(next_input)
    

    input_seq.append(predicted_price[0].tolist())

In [14]:
forecasted_prices = scaler.inverse_transform(input_seq[-forecast_days:])

In [ ]:
plt.figure(figsize=(12,6))
plt.title(f'Forecasted Stock Price of {stock} for Next {forecast_days} Days')
plt.xlabel('Days')
plt.ylabel('Close Price (USD)')
plt.plot(forecasted_prices, label='Forecasted Price')
plt.legend()
plt.show()